In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as pyplot
import seaborn as sns
import scipy as stats

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


Matplotlib created a temporary config/cache directory at /tmp/matplotlib-wnqrgbav because the default path (/home/user/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
path = "./files/iris.csv"

headers = []

# df = pd.read_csv(path, sep=";")
df = pd.read_csv("./files/imports-85.data")
# pd.set_option('display.max_rows', 500)

headers = ["symboling", "normalized-losses", "make", "fuel-type", "aspiration", "num-of-doors", "body-style",
           "drive-wheels", "engine-location", "wheel-base", "length", "width", "height", "curb-weight", "engine-type",
           "num-of-cylinders", "engine-size", "fuel-system", "bore", "stroke", "compression-ratio", "horsepower",
           "peak-rpm", "city-mpg", "highway-mpg", "price"]

df.columns = headers
df.info()
# df.describe(include="all")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          204 non-null    int64  
 1   normalized-losses  204 non-null    object 
 2   make               204 non-null    object 
 3   fuel-type          204 non-null    object 
 4   aspiration         204 non-null    object 
 5   num-of-doors       204 non-null    object 
 6   body-style         204 non-null    object 
 7   drive-wheels       204 non-null    object 
 8   engine-location    204 non-null    object 
 9   wheel-base         204 non-null    float64
 10  length             204 non-null    float64
 11  width              204 non-null    float64
 12  height             204 non-null    float64
 13  curb-weight        204 non-null    int64  
 14  engine-type        204 non-null    object 
 15  num-of-cylinders   204 non-null    object 
 16  engine-size        204 non

In [3]:
# Remove missing values
# print(df.head())
df.replace("?", np.nan, inplace=True)
for col in df.columns:
    if df[col].isna().values.any() == True:
        print(col + " " + str(df[col].isna().values.any()))


normalized-losses True
num-of-doors True
bore True
stroke True
horsepower True
peak-rpm True
price True


In [4]:
# normalized-losses
df["normalized-losses"].replace(np.nan, df[~df["normalized-losses"].isnull()]
                                ["normalized-losses"].astype("int").mean(), inplace=True)
df["normalized-losses"] = df["normalized-losses"].astype("int")
# num-of-doors
df = df[~df["num-of-doors"].isna()]
# bore
df_bore_finding = df[~df["bore"].isna()]
df_bore_finding["bore"] = df_bore_finding["bore"].astype("float")
df_mean = df_bore_finding.groupby(
    ["make", "body-style"], as_index=False).mean()
df_mean[df_mean["make"] == "mazda"][df_mean["body-style"] == "hatchback"]["bore"]

df["bore"].replace(np.nan, df_mean[df_mean["make"] == "mazda"]
                   [df_mean["body-style"] == "hatchback"]["bore"].max(), inplace=True)
df["bore"] = df["bore"].astype("float")
# stroke
df_bore_finding = df[~df["stroke"].isna()]
df_bore_finding["stroke"] = df_bore_finding["stroke"].astype("float")
df_mean = df_bore_finding.groupby(
    ["make", "body-style"], as_index=False).mean()
df_mean[df_mean["make"] == "mazda"][df_mean["body-style"] == "hatchback"]["stroke"]

df["stroke"].replace(np.nan, df_mean[df_mean["make"] == "mazda"]
                     [df_mean["body-style"] == "hatchback"]["stroke"].max(), inplace=True)
df["stroke"] = df["stroke"].astype("float")
# horsepower
df_hp = df[~df["horsepower"].isna()]
df_hp_mean = df_hp[df_hp['body-style'] == 'wagon'][df_hp['bore']
                                                   >= 3.46][df_hp['bore'] <= 3.50]['horsepower'].astype("int").mean()
df["horsepower"].replace(np.nan, df_hp_mean, inplace=True)
df["horsepower"] = df["horsepower"].astype("int")
df[df["make"] == "renault"]
# peak-rpm
df_hp = df[~df["peak-rpm"].isna()]
df_hp_mean = df_hp[df_hp['body-style'] == 'wagon'][df_hp['bore']
                                                   >= 3.46][df_hp['bore'] <= 3.50]['peak-rpm'].astype("int").mean()
df["peak-rpm"].replace(np.nan, df_hp_mean, inplace=True)
df["peak-rpm"] = df["peak-rpm"].astype("int")
# price
df[df['price'].isna()]
df_price = df[~df['price'].isna()]
df_price['price'] = df['price'].astype("float")
for index, row in df[df['price'].isna()].iterrows():
    row_mean = df_price[df_price["body-style"] == row["body-style"]
                        ][df_price["drive-wheels"] == row["drive-wheels"]]["price"].mean().round(2)
    df.loc[index, "price"] = row_mean

df["price"] = df["price"].astype("float")
df.describe()


/tmp/ipykernel_18732/3544775526.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bore_finding["bore"] = df_bore_finding["bore"].astype("float")
/tmp/ipykernel_18732/3544775526.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_mean[df_mean["make"] == "mazda"][df_mean["body-style"] == "hatchback"]["bore"]
/tmp/ipykernel_18732/3544775526.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df["bore"].replace(np.nan, df_mean[df_mean["make"] == "mazda"]
/tmp/ipykernel_18732/3544775526.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,symboling,normalized-losses,wheel-base,length,width,height,curb-weight,engine-size,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
count,202.000000,202.000000,202.000000,202.000000,202.000000,202.000000,202.000000,202.000000,202.000000,202.000000,202.000000,202.000000,202.000000,202.000000,202.000000,202.000000
mean,0.826733,121.871287,98.831683,174.139604,65.924257,53.755941,2557.965347,127.059406,3.327871,3.257228,10.098614,104.351485,5125.247525,25.193069,30.717822,13184.948515
std,1.243762,31.863566,6.013061,12.364039,2.151791,2.423638,523.854860,41.900456,0.272342,0.313024,3.897109,39.716911,478.706994,6.539383,6.886758,7936.072150
min,-2.000000,65.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,13.000000,16.000000,5118.000000
25%,0.000000,101.000000,94.500000,166.425000,64.125000,52.000000,2145.000000,97.000000,3.150000,3.110000,8.600000,70.000000,4800.000000,19.000000,25.000000,7775.000000
50%,1.000000,122.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,119.500000,3.310000,3.290000,9.000000,95.000000,5200.000000,24.000000,30.000000,10221.500000
75%,2.000000,137.000000,102.400000,183.400000,66.900000,55.500000,2947.750000,144.000000,3.587500,3.410000,9.400000,116.000000,5500.000000,30.000000,34.000000,16500.000000
max,3.000000,256.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,3.940000,4.170000,23.000000,288.000000,6600.000000,49.000000,54.000000,45400.000000


In [5]:
# After solving missing values
# print(df.head())
# df[~df.isna()]
for col in df.columns:
    if df[col].isna().values.any() == True:
        print(col + " " + str(df[col].isna().values.any()))

In [7]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df[["highway-mpg"]], df[["price"]], test_size = 0.3)

RidgeModel = Ridge(alpha=0.1)
RidgeModel.fit(x_train, y_train)
Yhat = RidgeModel.predict(x_test)
Yhat

array([[18132.73716456],
       [17301.24081461],
       [10649.27001502],
       [11480.76636497],
       [ 3997.29921544],
       [17301.24081461],
       [16469.74446466],
       [ 9817.77366508],
       [19795.72986445],
       [ 9817.77366508],
       [15638.24811471],
       [13975.25541482],
       [18964.2335145 ],
       [10649.27001502],
       [17301.24081461],
       [14806.75176476],
       [19795.72986445],
       [13143.75906487],
       [ 7323.28461523],
       [13143.75906487],
       [ 9817.77366508],
       [17301.24081461],
       [13143.75906487],
       [17301.24081461],
       [18132.73716456],
       [ 6491.78826528],
       [ 6491.78826528],
       [ 5660.29191534],
       [18132.73716456],
       [14806.75176476],
       [13143.75906487],
       [11480.76636497],
       [ 2334.30651554],
       [17301.24081461],
       [13975.25541482],
       [ -160.1825343 ],
       [-6812.15333388],
       [ 7323.28461523],
       [21458.72256435],
       [17301.24081461],


In [13]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import train_test_split

parameters1 = [
    {"alpha": [0.001, 0.1, 1, 10, 100, 1000, 10000, 100000, 1000000], "normalize": [True, False]}]

rr = Ridge()

z = df[["horsepower", "curb-weight", "engine-size", "highway-mpg"]]
y = df[["price"]]

# x_train, x_test, y_train, y_test = train_test_split(z, y, test_size=0.3)

Grid1 = GridSearchCV(rr, parameters1, cv=4)
Grid1.fit(z, y)
Grid1.best_estimator_
scores = Grid1.cv_results_
for param, mean_val in zip(scores["params"], scores["mean_test_score"]):
    print(param, "R^2 on test data:", mean_val,
          "R^2 on train data:")


/home/user/backup/PythonDataScience/env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(
/home/user/backup/PythonDataScience/env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing st

{'alpha': 0.001, 'normalize': True} R^2 on test data: 0.6680758018438628 R^2 on train data:
{'alpha': 0.001, 'normalize': False} R^2 on test data: 0.6675544391535139 R^2 on train data:
{'alpha': 0.1, 'normalize': True} R^2 on test data: 0.689486870108873 R^2 on train data:
{'alpha': 0.1, 'normalize': False} R^2 on test data: 0.6675550365304921 R^2 on train data:
{'alpha': 1, 'normalize': True} R^2 on test data: 0.6698162087671351 R^2 on train data:
{'alpha': 1, 'normalize': False} R^2 on test data: 0.667560463178483 R^2 on train data:
{'alpha': 10, 'normalize': True} R^2 on test data: 0.30730837881338524 R^2 on train data:
{'alpha': 10, 'normalize': False} R^2 on test data: 0.6676143309033684 R^2 on train data:
{'alpha': 100, 'normalize': True} R^2 on test data: 0.01568418984864295 R^2 on train data:
{'alpha': 100, 'normalize': False} R^2 on test data: 0.6681157318335136 R^2 on train data:
{'alpha': 1000, 'normalize': True} R^2 on test data: -0.029424676293149488 R^2 on train data:
{'a

/home/user/backup/PythonDataScience/env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(
/home/user/backup/PythonDataScience/env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The 